# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import pandas as pd
import numpy as np
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [9]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url)

In [10]:
assert data.status_code == 200

In [13]:
data = data.json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2564712125071,
 'week52high': 158.62,
 'week52low': 109.71,
 'week52highSplitAdjustOnly': 163.44,
 'week52lowSplitAdjustOnly': 110.39,
 'week52change': 0.3107007822926281,
 'sharesOutstanding': 16579981705,
 'float': 0,
 'avg10Volume': 70979079,
 'avg30Volume': 78524196,
 'day200MovingAvg': 140.2,
 'day50MovingAvg': 148.62,
 'employees': 149393,
 'ttmEPS': 5.2,
 'ttmDividendRate': 0.878951658983304,
 'dividendYield': 0.005980791948136528,
 'nextDividendDate': '',
 'exDividendDate': '2021-08-02',
 'nextEarningsDate': '2021-10-16',
 'peRatio': 29.49545926044014,
 'beta': 1.439688783260303,
 'maxChangePercent': 58.50537155638371,
 'year5ChangePercent': 4.498102980026187,
 'year2ChangePercent': 1.53959979595601,
 'year1ChangePercent': 0.2989047562068939,
 'ytdChangePercent': 0.1317222379426438,
 'month6ChangePercent': 0.134322289220279,
 'month3ChangePercent': 0.014893268584561503,
 'month1ChangePercent': 0.02026884046560812,
 'day30ChangePercent'

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [14]:
data['year1ChangePercent']

0.2989047562068939

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [7]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [43]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ],
        index=my_columns    
        ),
        ignore_index=True
        )

final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,158.12,0.511834,N/A
1,AAL,19.52,0.523552,N/A
2,AAP,240.94,0.527645,N/A
3,AAPL,152.05,0.305545,N/A
4,ABBV,109.73,0.360568,N/A
...,...,...,...,...
500,YUM,127.25,0.275891,N/A
501,ZBH,154.67,0.063328,N/A
502,ZBRA,531.42,0.751612,N/A
503,ZION,66.82,1.078853,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [44]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)

In [45]:
final_dataframe.reset_index(inplace=True)

In [46]:
final_dataframe = final_dataframe[:50]

In [47]:
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,148,DVN,42.37,3.567041,N/A
1,175,FANG,110.44,2.877813,N/A
2,317,MRO,17.05,2.86492,N/A
3,357,OXY,35.29,2.423545,N/A
4,275,LB,83.14,2.358551,N/A
5,299,MCHP,76.90,1.724615,N/A
6,251,IT,338.61,1.599911,N/A
7,195,FTNT,346.32,1.574798,N/A
8,410,SIVB,775.00,1.54558,N/A
9,160,EOG,95.00,1.529793,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [4]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the value of your portfolio:')
    try:
        float(portfolio_size)
      
    except ValueError:
        print("That's not a number! \nPlease try again:")
        portfolio_size = input('Enter the value of your portfolio:')
                
portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:1000000000
1000000000


In [59]:
position_size = float(portfolio_size)/ len(final_dataframe.index)

for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])
    
final_dataframe

C:\Users\SOLA\AppData\Local\Programs\Python\Python39-32\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,148,DVN,42.37,3.567041,4720
1,175,FANG,110.44,2.877813,1810
2,317,MRO,17.05,2.86492,11730
3,357,OXY,35.29,2.423545,5667
4,275,LB,83.14,2.358551,2405
5,299,MCHP,76.90,1.724615,2600
6,251,IT,338.61,1.599911,590
7,195,FTNT,346.32,1.574798,577
8,410,SIVB,775.00,1.54558,258
9,160,EOG,95.00,1.529793,2105


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [25]:
hqm_columns = [
    'Ticker',
    'Price',     
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)

In [26]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['price'],
            'N/A',
            data[symbol]['stats']['year1ChangePercent'],
            'N/A',
            data[symbol]['stats']['month6ChangePercent'],
            'N/A',
            data[symbol]['stats']['month3ChangePercent'],
            'N/A',
            data[symbol]['stats']['month1ChangePercent'],
            'N/A',
            'N/A'
        ],
        index=hqm_columns    
        ),
        ignore_index=True
        )

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,165.01,N/A,0.51692,N/A,0.163434,N/A,0.039234,N/A,-0.088639,N/A,N/A
1,AAL,19.41,N/A,0.530324,N/A,-0.093972,N/A,-0.099755,N/A,-0.085305,N/A,N/A
2,AAP,232.53,N/A,0.539964,N/A,0.179041,N/A,0.086249,N/A,0.084514,N/A,N/A
3,AAPL,154.49,N/A,0.312314,N/A,0.115363,N/A,0.002464,N/A,0.013164,N/A,N/A
4,ABBV,113.17,N/A,0.361215,N/A,0.002952,N/A,-0.066957,N/A,0.029189,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,131.78,N/A,0.280576,N/A,0.086773,N/A,0.03523,N/A,0.013033,N/A,N/A
501,ZBH,156.05,N/A,0.062772,N/A,-0.153351,N/A,-0.06072,N/A,0.001751,N/A,N/A
502,ZBRA,550.41,N/A,0.741504,N/A,0.062288,N/A,-0.038762,N/A,-0.047789,N/A,N/A
503,ZION,68.56,N/A,1.081367,N/A,0.250237,N/A,0.323569,N/A,0.121409,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [27]:
#hqm_dataframe1= hqm_dataframe.copy()

In [28]:
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

In [29]:
hqm_dataframe.head()

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,165.01,N/A,0.51692,N/A,0.163434,N/A,0.039234,N/A,-0.088639,N/A,N/A
1,AAL,19.41,N/A,0.530324,N/A,-0.093972,N/A,-0.099755,N/A,-0.085305,N/A,N/A
2,AAP,232.53,N/A,0.539964,N/A,0.179041,N/A,0.086249,N/A,0.084514,N/A,N/A
3,AAPL,154.49,N/A,0.312314,N/A,0.115363,N/A,0.002464,N/A,0.013164,N/A,N/A
4,ABBV,113.17,N/A,0.361215,N/A,0.002952,N/A,-0.066957,N/A,0.029189,N/A,N/A


In [30]:
hqm_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Ticker                         505 non-null    object 
 1   Price                          505 non-null    float64
 2   Number of Shares to Buy        505 non-null    object 
 3   One-Year Price Return          501 non-null    object 
 4   One-Year Return Percentile     505 non-null    object 
 5   Six-Month Price Return         501 non-null    object 
 6   Six-Month Return Percentile    505 non-null    object 
 7   Three-Month Price Return       501 non-null    object 
 8   Three-Month Return Percentile  505 non-null    object 
 9   One-Month Price Return         501 non-null    object 
 10  One-Month Return Percentile    505 non-null    object 
 11  HQM Score                      505 non-null    object 
dtypes: float64(1), object(11)
memory usage: 25.7+ KB


In [31]:
for col in hqm_dataframe.columns[3:11:2]:
    hqm_dataframe[col] = pd.to_numeric(pd.to_numeric(hqm_dataframe[col]))
    hqm_dataframe[col] = hqm_dataframe[col].map(lambda x: float(x))

In [32]:
time_periods = ['One-Year','Six-Month','Three-Month','One-Month']

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col],hqm_dataframe.loc[row, change_col])/100
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,165.01,N/A,0.516920,0.683168,0.163434,0.69901,0.039234,0.516832,-0.088639,0.035644,N/A
1,AAL,19.41,N/A,0.530324,0.69505,-0.093972,0.148515,-0.099755,0.087129,-0.085305,0.039604,N/A
2,AAP,232.53,N/A,0.539964,0.70495,0.179041,0.744554,0.086249,0.689109,0.084514,0.778218,N/A
3,AAPL,154.49,N/A,0.312314,0.447525,0.115363,0.59604,0.002464,0.370297,0.013164,0.427723,N/A
4,ABBV,113.17,N/A,0.361215,0.518812,0.002952,0.334653,-0.066957,0.144554,0.029189,0.534653,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,131.78,N/A,0.280576,0.405941,0.086773,0.548515,0.035230,0.508911,0.013033,0.423762,N/A
501,ZBH,156.05,N/A,0.062772,0.162376,-0.153351,0.069307,-0.060720,0.160396,0.001751,0.342574,N/A
502,ZBRA,550.41,N/A,0.741504,0.831683,0.062288,0.493069,-0.038762,0.20396,-0.047789,0.10495,N/A
503,ZION,68.56,N/A,1.081367,0.922772,0.250237,0.851485,0.323569,0.968317,0.121409,0.875248,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [33]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = [hqm_dataframe.loc[row, f'{time_period} Return Percentile'] for time_period in time_periods]
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,165.01,N/A,0.516920,0.683168,0.163434,0.69901,0.039234,0.516832,-0.088639,0.035644,0.483663
1,AAL,19.41,N/A,0.530324,0.69505,-0.093972,0.148515,-0.099755,0.087129,-0.085305,0.039604,0.242574
2,AAP,232.53,N/A,0.539964,0.70495,0.179041,0.744554,0.086249,0.689109,0.084514,0.778218,0.729208
3,AAPL,154.49,N/A,0.312314,0.447525,0.115363,0.59604,0.002464,0.370297,0.013164,0.427723,0.460396
4,ABBV,113.17,N/A,0.361215,0.518812,0.002952,0.334653,-0.066957,0.144554,0.029189,0.534653,0.383168
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,131.78,N/A,0.280576,0.405941,0.086773,0.548515,0.035230,0.508911,0.013033,0.423762,0.471782
501,ZBH,156.05,N/A,0.062772,0.162376,-0.153351,0.069307,-0.060720,0.160396,0.001751,0.342574,0.183663
502,ZBRA,550.41,N/A,0.741504,0.831683,0.062288,0.493069,-0.038762,0.20396,-0.047789,0.10495,0.408416
503,ZION,68.56,N/A,1.081367,0.922772,0.250237,0.851485,0.323569,0.968317,0.121409,0.875248,0.904455


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [34]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)

In [35]:
hqm_dataframe = hqm_dataframe[:50]

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [36]:
portfolio_input()

Enter the value of your portfolio:1000000000


In [37]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i,'Price'])
    
hqm_dataframe.reset_index(drop=True,inplace=True)

C:\Users\SOLA\AppData\Local\Programs\Python\Python39-32\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [38]:
hqm_dataframe.head()

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,DVN,41.46,482392,3.529980,0.992079,0.940339,0.990099,0.593856,0.990099,0.240427,0.982178,0.988614
1,MRO,17.12,1168224,2.906432,0.988119,0.575507,0.982178,0.467229,0.986139,0.314513,0.992079,0.987129
2,FANG,113.91,175577,2.916254,0.990099,0.458833,0.972277,0.406819,0.982178,0.255771,0.988119,0.983168
3,COP,79.16,252652,1.384788,0.966337,0.561658,0.980198,0.370054,0.976238,0.193857,0.964356,0.971782
4,OXY,35.60,561797,2.416067,0.986139,0.395362,0.964356,0.284165,0.948515,0.214391,0.976238,0.968812


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [39]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [40]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [41]:
column_formats = {
    'A':['Ticker',string_template],
    'B':['Price', dollar_template],     
    'C':['Number of Shares to Buy', integer_template],
    'D':['One-Year Price Return', percent_template],
    'E':['One-Year Return Percentile', percent_template],
    'F':['Six-Month Price Return', percent_template],
    'G':['Six-Month Return Percentile', percent_template],
    'H':['Three-Month Price Return', percent_template],
    'I':['Three-Month Return Percentile', percent_template],
    'J':['One-Month Price Return', percent_template],
    'K':['One-Month Return Percentile', percent_template],
    'L':['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [42]:
writer.save()